In [1]:
# Introduction

# When investigating astronomical objects, like active galactic nuclei (AGN), astronomers compare data about those objects from different telescopes at different wavelengths.
# This requires positional cross-matching to find the closest counterpart within a given radius on the sky.
# In this activity you'll cross-match two catalogues: one from a radio survey, the AT20G Bright Source Sample (BSS) catalogue and one from an optical survey, the SuperCOSMOS all-sky galaxy catalogue.
# The BSS catalogue lists the brightest sources from the AT20G radio survey while the SuperCOSMOS catalogue lists galaxies observed by visible light surveys. If we can find an optical match for our radio source, we are one step closer to working out what kind of object it is, e.g. a galaxy in the local Universe or a distant quasar.
# We've chosen one small catalogue (BSS has only 320 objects) and one large one (SuperCOSMOS has about 240 million) to demonstrate the issues you can encounter when implementing cross-matching algorithms.

# https://groklearning-cdn.com/modules/5JAX7dX9j4wdXKXRHwyECd/Ra_and_dec_demo.gif
# The positions of stars, galaxies and other astronomical objects are usually recorded in either equatorial or Galactic coordinates.


# Equatorial coordinates are fixed relative to the celestial sphere, so the positions are independent of when or where the observations took place. They are defined relative to the celestial equator (which is in the same plane as the Earth's equator) and the ecliptic (the path the sun traces throughout the year).
# A point on the celestial sphere is given by two coordinates:
# Right ascension: the angle from the vernal equinox to the point, going east along the celestial equator;
# Declination: the angle from the celestial equator to the point, going north (negative values indicate going south).
# The vernal equinox is the intersection of the celestial equator and the ecliptic where the ecliptic rises above the celestial equator going further east.


In [4]:
# Coordinates: Right Ascension

# Right ascension is often given in hours-minutes-seconds (HMS) notation, because it was convenient to calculate when a star would appear over the horizon. A full circle in HMS notation is 24 hours, which means 1 hour in HMS notation is equal to 15 degrees.
# Each hour is split into 60 minutes and each minute into 60 seconds.
# You can convert 23 hours, 12 minutes and 6 seconds (written as 23:12:06 or 23h12m06s) to degrees like this:

print(15*(23 + 12/60 + 6/(60*60)))

# Coordinates: Declination

# Declination, on the other hand, is traditionally recorded in degrees-minutes-seconds (DMS) notation. A full circle is 360 degrees, each degree has 60 arcminutes and each arcminute has 60 arcseconds.
# For example: 73 degrees, 21 arcminutes and 14.4 arcseconds (written 73:21:14.4 or 73° 21' 14.4" or 73d21m14.4s) can be converted to decimal degrees like this:

print(73 + 21/60 + 14.4/(60*60))

# With negative angles like -5° 31' 12" the negation applies to the whole angle, including arcminutes and arcseconds:

print(-1*(5 + 31/60 + 12/(60*60)))

# Note: arcminutes and minutes are different!
# The arcminutes and arcseconds in DMS are not the same as the minutes and seconds in HMS! A minute in HMS is equal to 15 arcminutes in DMS and a second is equal to 15 arcseconds.

348.025
73.354
-5.52


In [5]:
# Assignment: Convdert to Decimal Degrees

# Write two functions, one that converts right ascension from HMS to decimal degrees, called hms2dec, and another that converts declination from DMS to decimal degrees, called dms2dec.
# Right ascension is always an angle from 0 to 24 hours and declination is always an angle from -90° to +90°.

# Your hms2dec function should work like this:

# >>> hms2dec(23, 12, 6)
# 348.025

# And your dms2dec function should work like this:

# >>> dms2dec(22, 57, 18)
# 22.955

# It should also work with negative angles:

# >>> dms2dec(-66, 5, 5.1)
# -66.08475


# Write your hms2dec and dms2dec functions here

# Both the functions just use the equations seen in previous slides, but it's important to remember the difference between minutes and seconds in HMS and arcminutes and arcseconds in DMS.
# In DMS it's also important remember to keep track of the negative sign of the angle, if there is one. We've given you two different ways of doing this.



def hms2dec(ah, am, ase):
  return 15*(ah + am/60 + ase/(60**2))
  
def dms2dec(h, m, se):
  if h>0 :
    sign = 1
  else : 
    sign = -1
  return sign*(abs(h) + m/60 + se/(60**2))



# You can use this to test your function.
# Any code inside this `if` statement will be ignored by the automarker.
if __name__ == '__main__':
  # The first example from the question
  print(hms2dec(23, 12, 6))

  # The second example from the question
  print(dms2dec(22, 57, 18))

  # The third example from the question
  print(dms2dec(-66, 5, 5.1))
    
    
    
# Week 2 Assignment 1 completed

348.025
22.955
-66.08475


In [9]:
# Angular Distances

# To crossmatch two catalogues we need to compare the angular distance between objects on the celestial sphere.
# People loosely call this a "distance", but technically its an angular distance: the projected angle between objects as seen from Earth.
# If we have an object on the celestial sphere with right ascension and declination (alpha1, delta1), then the angular distance to another object with coordinates (alpha2, delta2) is:
# d = 2*arcsin(((sin(|delta1 - delta2|)/2)^2 + cos(delta1)*cos(delta2)*(sin(|alpha1 - aplha2|)/2)^2)^1/2)

# Angular distances have the same units as angles (degrees). There are other equations for calculating the angular distance but this one, called the haversine formula, is good at avoiding floating point errors when the two points are close together.



# Implementation

# We'll go through an example of how to implement the formula you saw on the previous slide using NumPy's trigonometric functions.
# First, let's break down the formula into smaller parts:
# 
# d = 2*arcsin((a+b)^1/2)
# a = (sin(|delta1 - delta2|)/2)^2
# b = cos(delta1)*cos(delta2)*(sin(|alpha1 - alpha2|)/2)^2
# 
# We can calculate b with NumPy's sin, cos and abs functions:
# 
# b = np.cos(d1)*np.cos(d2)*np.sin(np.abs(r1 - r2)/2)**2
# 
# Here, r1 and d1 are the coordinates of the first point (alpha1,delta1) and r2 and d2 similarly correspond to alpha2 and delta2
# a can be calculated in a similar way using just sin and abs. Once we have both a and b we can use numpy.arcsin to calculate d:

# import numpy as np
# d = 2*np.arcsin(np.sqrt(a + b))

# Using NumPy, the code works with individual or arrays of sources.

# Note
# Trig functions in most languages and libraries (including Python and NumPy) take angle arguments in units of radians, but the databases we're working with use angles of degrees.
# Fortunately, NumPy provides convenient conversion functions:

# a_rad = np.radians(a_deg)
# a_deg = np.degrees(a_rad)

# The variable a_deg is in units of degrees and a_rad is in radians.


In [10]:
# Assignment: Heavenly Angles

# Write a function called angular_dist that calculates the angular distance between any two points on the celestial sphere given their right ascension and declination.
# Your function should take arguments in decimal degrees and return the distance in decimal degrees too.
# Here's an example of how your function should work:

# >>> ra1, dec1 = 21.07, 0.1
# >>> ra2, dec2 = 21.15, 8.2
# >>> angular_dist(ra1, dec1, ra2, dec2)
# 8.1003923181465041

# Here’s another example:

# >>> angular_dist(10.3, -3, 24.3, -29)
# 29.208498180546595

# Implementation

# This function implements the haversine formula seen previously.
# We use NumPy in our solution for all the maths functions because we'll end up using NumPy for more complicated scripts involving this function later, but this could easily have been done using just the built in math module.
# It's important to note that neither of these two packages have trigonometric functions that work with degrees, so we'll always have to convert to and from radians during calculation.

# Write your angular_dist function here.
import numpy as np

def angular_dist(RA1, dec1, RA2, dec2):
  # Convert to radians
  r1 = np.radians(RA1)
  d1 = np.radians(dec1)
  r2 = np.radians(RA2)
  d2 = np.radians(dec2)

  a = np.sin(np.abs(d1 - d2)/2)**2
  b = np.cos(d1)*np.cos(d2)*np.sin(np.abs(r1 - r2)/2)**2

  angle = 2*np.arcsin(np.sqrt(a + b))
    
  # Convert back to degrees
  return np.degrees(angle)



# You can use this to test your function.
# Any code inside this `if` statement will be ignored by the automarker.
if __name__ == '__main__':
  # Run your function with the first example in the question.
  print(angular_dist(21.07, 0.1, 21.15, 8.2))

  # Run your function with the second example in the question
  print(angular_dist(10.3, -3, 24.3, -29))


8.100392318146504
29.208498180546595


In [11]:
# Loadin Catalogues

# # Before we can crossmatch our two catalogues we first have to import the raw data. Every astronomy catalogue tends to have its own unique format so we'll need to look at how to do this with each one individually.
# We'll look at the AT20G bright source sample survey first. The raw data we'll be using is the file table2.dat from this page in the VizieR archives, but we'll use the filename bss.dat from now on.
# Every catalogue in VizieR has a detailed README file that gives you the exact format of each table in the catalogue.

# The full catalogue of bright radio sources contains 320 objects. The first few rows look like this (scroll right to see it all):


# bss.dat

#   1  00 04 35.65 -47 36 19.1   0.87 0.04 0.97 0.06  0.90 0.04                0.995 0.030            17.63 Q 1.F.11.C  PKS 0002-478
#   2  00 10 35.92 -30 27 48.3   0.74 0.03 0.72 0.04  0.63 0.03  0.315 0.009   0.419 0.013 1.19  La01 19.59 Q 1.F.11..  PKS 0008-307
#   3* 00 11 01.27 -26 12 33.1   0.64 0.07 0.82 0.07  0.69 0.03  0.210 0.006               1.096 Wr83 19.53 Q 4.F.44.C  PKS 0008-264

# The catalogue is organised in fixed-width columns, with the format of the columns being:

# 1: Object catalogue ID number (sometimes with an asterisk)
# 2-4: Right ascension in HMS notation
# 5-7: Declination in DMS notation
# 8-: Other information, including spectral intensities

# We only need coordinates for crossmatching. We can load specific columns with the usecols argument in NumPy's loadtxt function:

# import numpy as np
# cat = np.loadtxt('bss.dat', usecols=range(1, 7))
# print(cat[0])

# We've skipped the ID column, since the ID number is always the same as the row number.

# Fixed-width columns and loadtxt
# loadtxt does not work for fixed-width columns if values are missing. Since there are no missing ID, RA and dec values it is fine for loading the first few columns of the BSS catalogue.

# SuperCOSMOS all-sky catalogue

# The SuperCOSMOS all-sky catalogue is a catalogue of galaxies generated from several visible light surveys.
# The original data is available on this page in a package called SCOS_XSC_mCl1_B21.5_R20_noStepWedges.csv.gz. Because this catalogue is so large, we've cut it down for these activities. The cut down version of the file will be named super.csv.
# The first few lines of super.csv look like this:

# super.csv
# RA,Dec,sigRA,sigDec,epoch,muAcosD,muD,sigMuAcosD,sigMuD,chi2,classMagB,classMagR1,classMagR2,classMagI,meanClass,classB,classR1,classR2,classI,ellipB,ellipR1,ellipR2,ellipI,qualB,qualR1,qualR2,qualI
# 1.0583407,-52.9162402,1.2605071E-05,1.3178029E-05,1990.9344,-14.794838,-22.16756,7.242738,7.881182,5.027039,14.072,12.997,13.293,12.74,1,1,1,1,1,0.182453,0.234902,0.213206,0.19472,16,16,16,16
# 2.6084425,-41.5005753,2.0626481E-05,2.0626481E-05,1990.0508,-1.144597,-0.50977,10.397644,11.014809,0.245407,18.84,18.834,18.387,18.929,2,2,2,2,2,0.106605,0.112284,0.137899,0.091846,0,0,0,0
# 2.7302499,-27.706955,1.9480565E-05,1.8334649E-05,1982.9619,8.661513,1.872858,11.983931,11.506061,0.281173,19.188,18.723,18.74,18.993,2,2,2,2,1,0.117095,0.174671,0.020132,0.280859,0,0,0,0


# The SuperCOSMOS all-sky catalogue is a catalogue of galaxies generated from several visible light surveys.
# The original data is available on this page in a package called SCOS_XSC_mCl1_B21.5_R20_noStepWedges.csv.gz. Because this catalogue is so large, we've cut it down for these activities. The cut down version of the file will be named super.csv.
# The first few lines of super.csv look like this:

# super.csv
# RA,Dec,sigRA,sigDec,epoch,muAcosD,muD,sigMuAcosD,sigMuD,chi2,classMagB,classMagR1,classMagR2,classMagI,meanClass,classB,classR1,classR2,classI,ellipB,ellipR1,ellipR2,ellipI,qualB,qualR1,qualR2,qualI
# 1.0583407,-52.9162402,1.2605071E-05,1.3178029E-05,1990.9344,-14.794838,-22.16756,7.242738,7.881182,5.027039,14.072,12.997,13.293,12.74,1,1,1,1,1,0.182453,0.234902,0.213206,0.19472,16,16,16,16
# 2.6084425,-41.5005753,2.0626481E-05,2.0626481E-05,1990.0508,-1.144597,-0.50977,10.397644,11.014809,0.245407,18.84,18.834,18.387,18.929,2,2,2,2,2,0.106605,0.112284,0.137899,0.091846,0,0,0,0
# 2.7302499,-27.706955,1.9480565E-05,1.8334649E-05,1982.9619,8.661513,1.872858,11.983931,11.506061,0.281173,19.188,18.723,18.74,18.993,2,2,2,2,1,0.117095,0.174671,0.020132,0.280859,0,0,0,0


# The catalogue uses a comma-separated value (CSV) format. Aside from the first row, which contains column labels, the format is:
# 1: Right ascension in decimal degrees
# 2: Declination in decimal degrees
# 3: Other data, including magnitude and apparent shape

# So now when loading this file in, we have to tell np.loadtxt to skip the first row and treat the commas as delimiters:

# import numpy as np
# cat = np.loadtxt('super.csv', delimiter=',', skiprows=1, usecols=[0, 1])
# print(cat)


In [12]:
# Assignment: Playing with Space cats

# Write import_bss and import_super functions that import the AT20G BSS and SuperCOSMOS catalogues from the files bss.dat and super.csv as described in the previous slides.
# Each function should return a list of tuples containing the object's ID (an integer) and the coordinates in degrees. The object ID should be the row of the object in the catalogue, starting at 1.
# Your functions should work like this:

# >>> import_bss()
# [(1, 1.1485416666666666, -47.60530555555556), (2, 2.6496666666666666, -30.463416666666667), (3, 2.7552916666666665, -26.209194444444442)]
# >>> import_super()
# [(1, 1.0583407, -52.916240199999997), (2, 2.6084425000000002, -41.500575300000001), (3, 2.7302499, -27.706955000000001)]

# The files bss.dat and super.csv contain only the first 3 rows of the respective full catalogues, but your function should work with any number of entries in either catalogue.

# The previous slides gave you the code necessary to load in the coordinate data for AT20G BSS with loadtxt:
# data = np.loadtxt('bss.dat', usecols=range(1, 7))
# SuperCOSMOS needs the skiprows argument:
# data = np.loadtxt('super.csv', delimiter=',', skiprows=1, usecols=(0, 1))

# For AT20G BSS, there is the additional step of converting the HMS format of the RA columns (0-2) and the DMS format of the declination columns (3-5) into decimal degrees using the hms2dec and dms2dec functions you developed in previous questions.


# Write your import_bss function here.
# Write your import_bss function here.
import numpy as np

def hms2dec(hr, m, s):
  dec = hr + m/60 + s/3600
  return dec*15

def dms2dec(d, m, s):
  sign = d/abs(d)
  dec = abs(d) + m/60 + s/3600
  return sign*dec

def import_bss():
  res = []
  data = np.loadtxt('bss.dat', usecols=range(1, 7))
  for i, row in enumerate(data, 1):
    res.append((i, hms2dec(row[0], row[1], row[2]), dms2dec(row[3], row[4], row[5])))
  return res

def import_super():
  data = np.loadtxt('super.csv', delimiter=',', skiprows=1, usecols=(0, 1))
  res = []
  for i, row in enumerate(data, 1):
    res.append((i, row[0], row[1]))
  return res



# You can use this to test your function.
# Any code inside this `if` statement will be ignored by the automarker.
if __name__ == '__main__':
  # Output of the import_bss and import_super functions
  bss_cat = import_bss()
  super_cat = import_super()
  print(bss_cat)
  print(super_cat)

[(1, 1.1485416666666666, -47.60530555555556), (2, 2.6496666666666666, -30.463416666666667), (3, 2.7552916666666665, -26.209194444444442)]
[(1, 1.0583407, -52.9162402), (2, 2.6084425, -41.5005753), (3, 2.7302499, -27.706955)]


In [13]:
# Assignment : Look Around You


# Write a find_closest function that takes a catalogue and the position of a target source (a right ascension and declination) and finds the closest match for the target source in the catalogue.
# Your function should return the ID of the closest object and the distance to that object.
# The right ascension and declination are in degrees. The catalogue list has been loaded by import_bss from the previous question. The full 320 object BSS catalogue is contained in bss.dat for you to test your code on.
# Here's an example of how your function should work:

# >>> cat = import_bss()
# >>> find_closest(cat, 175.3, -32.5)
# (156, 3.7670580226469053)

# And here's another example:

# >>> cat = import_bss()
# >>> find_closest(cat, 32.2, 40.7)
# (26, 57.729135775621295)

# Implementation

# In this solution, we go through each item in the list and pull out its individual RA and declination and use angular_dist to find the distance from the given point to the current object.
# The critical part is the loop over the catalogue and the comparison with the distance of the closest object so far:

# min_dist = np.inf
# min_id = None
# for id1, ra2, dec1 in cat:
#   dist = angular_dist(ra1, dec1, ra, dec)
#   if dist < min_dist:
#     min_id = id1
#     min_dist = dist

# Here we only keep track of the closest object so far and throw it away if the current object in the loop is closer.

# Write your find_closest function here
import numpy as np

def hms2dec(hr, m, s):
  dec = hr + m/60 + s/3600
  return dec*15

def dms2dec(d, m, s):
  sign = d/abs(d)
  dec = abs(d) + m/60 + s/3600
  return sign*dec

def import_bss():
  res = []
  data = np.loadtxt('bss1.dat', usecols=range(1, 7))
  for i, row in enumerate(data, 1):
    res.append((i, hms2dec(row[0], row[1], row[2]), dms2dec(row[3], row[4], row[5])))
  return res

def import_super():
  data = np.loadtxt('super.csv', delimiter=',', skiprows=1, usecols=[0, 1])
  res = []
  for i, row in enumerate(data, 1):
    res.append((i, row[0], row[1]))
  return res

def angular_dist(ra1, dec1, ra2, dec2):
  # Convert to radians
  r1 = np.radians(ra1)
  d1 = np.radians(dec1)
  r2 = np.radians(ra2)
  d2 = np.radians(dec2)

  a = np.sin(np.abs(d1 - d2)/2)**2
  b = np.cos(d1)*np.cos(d2)*np.sin(np.abs(r1 - r2)/2)**2

  angle = 2*np.arcsin(np.sqrt(a + b))
    
  # Convert back to degrees
  return np.degrees(angle)

def find_closest(cat, ra, dec):
  min_dist = np.inf
  min_id = None
  for id1, ra1, dec1 in cat:
    dist = angular_dist(ra1, dec1, ra, dec)
    if dist < min_dist:
      min_id = id1
      min_dist = dist
    
  return min_id, min_dist




# You can use this to test your function.
# Any code inside this `if` statement will be ignored by the automarker.
if __name__ == '__main__':
  cat = import_bss()
  
  # First example from the question
  print(find_closest(cat, 175.3, -32.5))

  # Second example in the question
  print(find_closest(cat, 32.2, 40.7))

(156, 3.7670580226469053)
(26, 57.729135775621295)


In [23]:
# A full cross-matching program

# You now have all the tools necessary to crossmatch the BSS and SuperCOSMOS catalogues. In the next problem you'll put it all together to see how many of the bright radio sources in the BSS catalogue have a counterpart in the SuperCOSMOS catalogue. The process you should follow is:

# 1. Select an object from the BSS catalogue;
# 2. Go through all the objects in SuperCOSMOS and find the closest one to the BSS object;
# 3. If the objects are close enough, record the match;
# 4. Repeat 1-3 for all the other objects in the BSS catalogue.

# In step 3, if the closest object isn't within a given distance then it's unlikely that the two objects are actually counterparts, and it's more likely that they just happen to be nearby.

# The given distance you choose depends on the uncertainty of the measured object positions in each catalogue.


# Assignment : Matchup Program

# Write a crossmatch function that crossmatches two catalogues within a maximum distance. It should return a list of matches and non-matches for the first catalogue against the second.
# The list of matches contains tuples of the first and second catalogue object IDs and their distance. The list of non-matches contains the unmatched object IDs from the first catalogue only. Both lists should be ordered by the first catalogue's IDs.
# The BSS and SuperCOSMOS catalogues will be given as input arguments, each in the format you’ve seen previously. The maximum distance is given in decimal degrees.
# Here's how crossmatch should work on our sample catalogues with a maximum distance of 40 arcseconds:

# bss_cat = import_bss()
# super_cat = import_super()
# max_dist = 40/3600
# matches, no_matches = crossmatch(bss_cat, super_cat, max_dist)
# print(matches[:3])
# print(no_matches[:3])
# print(len(no_matches))

# [(1, 2, 0.00010988610938710059), (2, 4, 0.00076498459672424946), (3, 5, 0.00020863352870707666)]
# [5, 6, 11]
# 9

# Only 9 objects have no match. Let's try a 5 arcsecond maximum:

# bss_cat = import_bss()
# super_cat = import_super()
# max_dist = 5/3600
# matches, no_matches = crossmatch(bss_cat, super_cat, max_dist)
# print(matches[:3])
# print(no_matches[:3])
# print(len(no_matches))

# [(1, 2, 0.00010988610938710059), (2, 4, 0.00076498459672424946), (3, 5, 0.00020863352870707666)]
# [5, 6, 11]
# 40

# Now 40 objects have no match with the tighter search radius.




# Assignment Submission:
# This is quite similar to the solution to the previous problem, except now we have an extra loop to go through all the SuperCOSMOS objects for each BSS object. At the end of the loop over SuperCOSMOS objects the closest object is found and if it's closer than the maximum radius then a match is recorded. Otherwise, a no-match is recorded.

# Write your crossmatch function here.
# Write your crossmatch function here.
import numpy as np

def hms2dec(hr, m, s):
  dec = hr + m/60 + s/3600
  return dec*15

def dms2dec(d, m, s):
  sign = d/abs(d)
  dec = abs(d) + m/60 + s/3600
  return sign*dec

def import_bss():
  res = []
  data = np.loadtxt('bss1.dat', usecols=range(1, 7))
  for i, row in enumerate(data, 1):
    res.append((i, hms2dec(row[0], row[1], row[2]), dms2dec(row[3], row[4], row[5])))
  return res

def import_super():
  data = np.loadtxt('super1.csv', delimiter=',', skiprows=1, usecols=[0, 1])
  res = []
  for i, row in enumerate(data, 1):
    res.append((i, row[0], row[1]))
  return res

def angular_dist(RA1, dec1, RA2, dec2):
    # Convert to radians
    r1 = np.radians(RA1)
    d1 = np.radians(dec1)
    r2 = np.radians(RA2)
    d2 = np.radians(dec2)
    
    deltar = np.abs(r1 - r2)
    deltad = np.abs(d1 - d2)
    angle = 2*np.arcsin(np.sqrt(np.sin(deltad/2)**2 
                        + np.cos(d1)*np.cos(d2)*np.sin(deltar/2)**2))
    
    # Convert back to degrees
    return np.degrees(angle)

def crossmatch(cat1, cat2, max_radius):
    matches = []
    no_matches = []
    for id1, ra1, dec1 in cat1:
        closest_dist = np.inf
        closest_id2 = None
        for id2, ra2, dec2 in cat2:
            dist = angular_dist(ra1, dec1, ra2, dec2)
            if dist < closest_dist:
                closest_id2 = id2
                closest_dist = dist
        
        # Ignore match if it's outside the maximum radius
        if closest_dist > max_radius:
            no_matches.append(id1)
        else:
            matches.append((id1, closest_id2, closest_dist))

    return matches, no_matches



# You can use this to test your function.
# Any code inside this `if` statement will be ignored by the automarker.
if __name__ == '__main__':
  bss_cat = import_bss()
  super_cat = import_super()

  # First example in the question
  max_dist = 40/3600
  matches, no_matches = crossmatch(bss_cat, super_cat, max_dist)
  print(matches[:3])
  print(no_matches[:3])
  print(len(no_matches))

  # Second example in the question
  max_dist = 5/3600
  matches, no_matches = crossmatch(bss_cat, super_cat, max_dist)
  print(matches[:3])
  print(no_matches[:3])
  print(len(no_matches))
    
  max_dist = 1/3600
  matches, no_matches = crossmatch(bss_cat, super_cat, max_dist)
  print(matches[:3])
  print(no_matches[:3])
  print(len(no_matches))


[(1, 2, 0.00010988610938710059), (2, 4, 0.0007649845967242495), (3, 5, 0.00020863352870707666)]
[5, 6, 11]
151
[(1, 2, 0.00010988610938710059), (2, 4, 0.0007649845967242495), (3, 5, 0.00020863352870707666)]
[5, 6, 11]
197
[(1, 2, 0.00010988610938710059), (3, 5, 0.00020863352870707666), (4, 6, 0.00012867299967083928)]
[2, 5, 6]
236


In [28]:

import numpy as np

def hms2dec(hr, m, s):
  dec = hr + m/60 + s/3600
  return dec*15

def dms2dec(d, m, s):
  sign = d/abs(d)
  dec = abs(d) + m/60 + s/3600
  return sign*dec

def import_bss():
  res = []
  data = np.loadtxt('bss2.dat', usecols=range(1, 7))
  for i, row in enumerate(data, 1):
    res.append((i, hms2dec(row[0], row[1], row[2]), dms2dec(row[3], row[4], row[5])))
  return res

def import_super():
  data = np.loadtxt('super1.csv', delimiter=',', skiprows=1, usecols=[0, 1])
  res = []
  for i, row in enumerate(data, 1):
    res.append((i, row[0], row[1]))
  return res

def angular_dist(RA1, dec1, RA2, dec2):
    # Convert to radians
    r1 = np.radians(RA1)
    d1 = np.radians(dec1)
    r2 = np.radians(RA2)
    d2 = np.radians(dec2)
    
    deltar = np.abs(r1 - r2)
    deltad = np.abs(d1 - d2)
    angle = 2*np.arcsin(np.sqrt(np.sin(deltad/2)**2 
                        + np.cos(d1)*np.cos(d2)*np.sin(deltar/2)**2))
    
    # Convert back to degrees
    return np.degrees(angle)

def crossmatch(cat1, cat2, max_radius):
    matches = []
    no_matches = []
    for id1, ra1, dec1 in cat1:
        closest_dist = np.inf
        closest_id2 = None
        for id2, ra2, dec2 in cat2:
            dist = angular_dist(ra1, dec1, ra2, dec2)
            if dist < closest_dist:
                closest_id2 = id2
                closest_dist = dist
        
        # Ignore match if it's outside the maximum radius
        if closest_dist > max_radius:
            no_matches.append(id1)
        else:
            matches.append((id1, closest_id2, closest_dist))

    return matches, no_matches



# You can use this to test your function.
# Any code inside this `if` statement will be ignored by the automarker.
if __name__ == '__main__':
  bss_cat = import_bss()
  super_cat = import_super()

  # First example in the question
  max_dist = 40/3600
  matches, no_matches = crossmatch(bss_cat, super_cat, max_dist)
  print(matches[:3])
  print(no_matches[:3])
  print(len(no_matches))

  # Second example in the question
  max_dist = 5/3600
  matches, no_matches = crossmatch(bss_cat, super_cat, max_dist)
  print(matches[:3])
  print(no_matches[:3])
  print(len(no_matches))
  
  # Third example in the question  
  max_dist = 0.05/3600
  matches, no_matches = crossmatch(bss_cat, super_cat, max_dist)
  print(matches[:3])
  print(no_matches[:3])
  print(len(no_matches))


# Congratulations, you've finished this set of activities.
# If you've still got questions about any of the content, head to the forums to discuss with your fellow learners.
# Now head back to Coursera for the next lecture.

[(1, 2, 0.00010988610938710059), (2, 4, 0.0007649845967242495), (3, 5, 0.00020863352870707666)]
[5, 6, 11]
9
[(1, 2, 0.00010988610938710059), (2, 4, 0.0007649845967242495), (3, 5, 0.00020863352870707666)]
[5, 6, 11]
40
[(130, 224, 4.268856487954769e-06)]
[1, 2, 3]
159
